In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
df_train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')

In [3]:
y_train = df_train['label']
x_train = df_train.drop(['label'], axis = 1)

In [4]:
x_train = x_train / 255
test = test / 255

In [5]:
x_train = x_train.values.reshape(-1, 28, 28, 1)
y_train = y_train.T
test = test.values.reshape(-1, 28, 28, 1)
x_train.shape

(42000, 28, 28, 1)

In [6]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [7]:
def convolutional_model(input_shape):
    
    input_img = tf.keras.Input(shape=input_shape)
    
    Z1 = tf.keras.layers.Conv2D(filters= 8, kernel_size= 4, strides= (1,1), padding='same')(input_img)
    
    A1 = tf.keras.layers.ReLU()(Z1)
    
    P1 = tf.keras.layers.MaxPool2D(pool_size=(8, 8), strides=(8, 8), padding='same')(A1)
    
    Z2 = tf.keras.layers.Conv2D(filters= 16, kernel_size= 2, strides= (1,1), padding='same')(P1)
    
    A2 = tf.keras.layers.ReLU()(Z2)
    
    P2 = tf.keras.layers.MaxPool2D(pool_size=(4, 4), strides=(4, 4), padding='same')(A2)
    
    F = tf.keras.layers.Flatten()(P2)
    
    outputs = tf.keras.layers.Dense(units= 10 , activation='softmax')(F)
    
    model = tf.keras.Model(inputs=input_img, outputs=outputs)
    
    return model

In [8]:
conv_model = convolutional_model((28, 28, 1))
conv_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
conv_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 8)         136       
_________________________________________________________________
re_lu (ReLU)                 (None, 28, 28, 8)         0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 8)           0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 16)          528       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 4, 4, 16)          0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 16)          0     

2022-03-06 08:50:36.503411: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-06 08:50:36.600708: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-06 08:50:36.601472: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-06 08:50:36.602582: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [9]:
history = conv_model.fit(x_train, y_train, epochs=20, validation_data = (x_val, y_val))

2022-03-06 08:50:38.845892: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-03-06 08:50:40.136215: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1050/1050 [==============================] - 9s 3ms/step - loss: 1.2189 - accuracy: 0.6539 - val_loss: 0.5462 - val_accuracy: 0.8388
Epoch 2/20
1050/1050 [==============================] - 3s 3ms/step - loss: 0.4389 - accuracy: 0.8670 - val_loss: 0.3706 - val_accuracy: 0.8807
Epoch 3/20
1050/1050 [==============================] - 3s 3ms/step - loss: 0.3350 - accuracy: 0.8977 - val_loss: 0.2890 - val_accuracy: 0.9101
Epoch 4/20
1050/1050 [==============================] - 3s 3ms/step - loss: 0.2845 - accuracy: 0.9124 - val_loss: 0.2584 - val_accuracy: 0.9196
Epoch 5/20
1050/1050 [==============================] - 3s 3ms/step - loss: 0.2523 - accuracy: 0.9230 - val_loss: 0.2252 - val_accuracy: 0.9311
Epoch 6/20
1050/1050 [==============================] - 3s 3ms/step - loss: 0.2320 - accuracy: 0.9289 - val_loss: 0.2066 - val_accuracy: 0.9379
Epoch 7/20
1050/1050 [==============================] - 3s 3ms/step - loss: 0.2150 - accuracy: 0.9337 - val_loss: 0.2005 - val_accuracy: 0.9375
Epo

In [10]:
prediction = []
for i in range(len(test)):
    prediction.append(conv_model.predict(test[i].reshape(1,28,28,1)).argmax())
submissions=pd.DataFrame({"ImageId": list(range(1,len(prediction)+1)),"Label": prediction})
submissions.to_csv("submission.csv", index=False, header=True)